In [1]:
!pip install -q pydub

import os
from pathlib import Path
from pydub import AudioSegment

In [34]:
# Config – adjust as needed
INPUT_DIR = "temp"                    # folder with original audio files
OUTPUT_DIR = "final"      # output folder
START_OFFSET_SEC = 10.0             # where to start the kept segment
DURATION_SEC = 20.0                # length of kept segment
SUPPORTED_EXTS = (".wav", ".mp3", ".flac", ".ogg", ".m4a")

In [35]:
def trim_audios(
    input_dir: str = INPUT_DIR,
    output_dir: str = OUTPUT_DIR,
    start_offset_sec: float = START_OFFSET_SEC,
    duration_sec: float = DURATION_SEC,
    exts=SUPPORTED_EXTS,
):
    """
    For each audio file in input_dir:
      - If shorter than duration_sec: copy unchanged to output_dir.
      - If longer: keep segment [start_offset_sec, start_offset_sec + duration_sec]
        (adjusted if it would run past the end), and save to output_dir.
    Logs what it does for each file.
    """
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    start_ms = int(start_offset_sec * 1000)
    dur_ms = int(duration_sec * 1000)

    total = 0
    trimmed = 0
    copied = 0

    for path in sorted(input_dir.iterdir()):
        if not path.is_file():
            continue
        if path.suffix.lower() not in exts:
            continue

        total += 1
        audio = AudioSegment.from_file(path)
        length_ms = len(audio)
        length_sec = length_ms / 1000.0

        out_path = output_dir / path.name

        if length_sec <= duration_sec:
            # Shorter than threshold: copy unchanged
            audio.export(out_path, format=path.suffix.lstrip("."))
            copied += 1
            print(f"[COPY]  {path.name}: {length_sec:.2f}s <= {duration_sec:.2f}s -> unchanged")
            continue

        # Compute segment boundaries
        seg_start = start_ms
        seg_end = start_ms + dur_ms

        # If segment would go past the end, slide it back
        if seg_end > length_ms:
            seg_end = length_ms
            seg_start = max(0, seg_end - dur_ms)

        trimmed_audio = audio[seg_start:seg_end]
        trimmed_audio.export(out_path, format=path.suffix.lstrip("."))
        trimmed += 1
        print(
            f"[TRIM]  {path.name}: {length_sec:.2f}s -> "
            f"{seg_start/1000:.2f}-{seg_end/1000:.2f}s "
            f"({len(trimmed_audio)/1000:.2f}s)"
        )

    print("\nSummary:")
    print(f"  Total audio files: {total}")
    print(f"  Trimmed:           {trimmed}")
    print(f"  Copied unchanged:  {copied}")


In [36]:
trim_audios()

[TRIM]  train_station_ambience.wav: 313.00s -> 10.00-30.00s (20.00s)

Summary:
  Total audio files: 1
  Trimmed:           1
  Copied unchanged:  0
